## Notes

In [ ]:
#vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
# These sections are encouraged to be altered! (Of course, you can change all of the code!)
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

exno and figno are the excercise number and figure number, each of which can be viewed in the original version of this file (chile2010b.ipynb)

This plots in this file do not currently appear to show velocity. The same incorrect values can be found when running the orgignal version of this file.

## Notebook setup

In [ ]:
%pylab inline

In [ ]:
from clawpack.clawutil import nbtools
from clawpack.visclaw import animation_tools
from IPython.display import HTML
from PIL import Image
from os import path, mkdir, remove
from shutil import rmtree, copyfile
from glob import glob
from distutils.dir_util import copy_tree

In [ ]:
nbtools.make_exe(new=True,verbose=False)

Check save path

In [ ]:
if path.isdir('_saved') == False:
    mkdir('_saved')
elif path.isdir('_saved/_plots') == False:
    mkdir('_saved/_plots')
elif path.isdir('_saved/_anim') == False:
    mkdir('_saved/_anim')

Clear old data files and diretories

In [ ]:
for k in range(1,11):
    try:
        remove('setrun_{}.py'.format(k))
        remove('plot_output_{}.txt'.format(k))
        remove('run_output_{}.txt'.format(k))
        rmtree('_plots_{}'.format(k))
        rmtree('_output_{}'.format(k))
    except Exception:
        pass

## Functions

### Initiate data compilation

Initiate changes to be written to the *.data files

In [ ]:
def init_comp(exno, anim123 = False, plots = False, gtype = 'stationary', out = False):
    
    if gtype == 'lagrangian':
        print('Warning: Only "travel" plots can be shown correctly on "lagrangian" gauge type')
    else:
        print('Warning: "travel" plots cannot be shown correctly on "stationary" gauge type')
        
    if plots == True:
        comp_manns = range(1, amount_manns + 1)
    # If user is not going to plot curves, only 2 manning's values are neccessary to be compiled
    elif plots == False:
        comp_manns = anim_manns

    for element in comp_manns:
        mod = importlib.import_module("setrun_{}".format(element))
        rundata = mod.setrun()
        # Changes to the original setrun.py
        #---------------------------------------
        # Change Mannings
        rundata.geo_data.manning_coefficient = manns[element - 1]
        # Change output times:
        rundata.clawdata.output_style = exno #This is the difference from ex1 to ex2 in the animation
        rundata.clawdata.output_times = linspace(3.5,8,19) * 3600.
            # Change gauge location:
        from clawpack.amrclaw.data import GaugeData
        rundata.replace_data("gaugedata", GaugeData())  # clear old gauge data
        rundata.gaugedata.gauges = []   # empty list
        rundata.gaugedata.gtype = gtype    
        
        if anim123 == True:
            gplot_amount = 2
        else:
            gplot_amount = len(dict_gauge)
        for j in range(gplot_amount):
            rundata.gaugedata.gauges.append([det[j][0], det[j][1], det[j][2], det[j][3], det[j][4]]) # not currently close to epicentre gauge
        #---------------------------------------
        compiler(rundata, element, comp_manns, out)
    if plots == False:
        return anim_manns

### Compile data

Write (any changes made by the user to) \*.data files, and then run the code from these files to create fortran data and images to be used in animation. 

In [ ]:
def compiler(rundata, element, comp_manns, out = False):
    # create *.data files from parameters in setrun.py
    rundata.write()
    # Run the code from the *.data files
    nbtools.make_output_and_plots(label="_{}".format(element), verbose = out)
    print("{:.0%} complete".format((comp_manns.index(element) + 1)/(len(comp_manns))))

### Show animation

In [ ]:
def show_anim(anim, form = 5):
    if form == 5:
        html_version = HTML(anim.to_html5_video())
    else:
        if form != 'js':
            print('format not recognised, defaulting to jshtml')
        html_version = HTML(anim.to_jshtml())

    return html_version

### Figures to animate

Concatenate frames to be used in animation, then creating the user-facing animation. 

In [ ]:
def stitch_anim(anim_area, form = 5):
    if anim_area == 'Full map':
        frames = 5
        exno = 1
        figno = 0
        extra = ''
    
    elif anim_area == 'North west coast':
        frames = 19
        exno = 2
        figno = 1
        extra = 0
        
    else:
        print ("Error : Check the animation area was entered correctly")
    
    old_frames = glob('_plots/*')
    for f in old_frames:
        remove(f) 
        
    for a in range(exno - 1 ,frames + 1):
        im = Image.open('_plots_{}/frame{:04d}fig{}.png'.format(anim_manns[0],a, figno))
        im_alt = Image.open('_plots_{}/frame{:04d}fig{}.png'.format(anim_manns[1], a, figno))
        dst = Image.new('RGB', (im.width + im_alt.width, im.height))
        dst.paste(im, (0, 0))
        dst.paste(im_alt, (im.width, 0))
        dst.save('_plots/frame{:04d}fig{}.png'.format(a, figno))
        

    anim = animation_tools.animate_from_plotdir('_plots', figno)
    
    anim_path = "_saved/_anim/manns_[{},{}]/{}".format(choose_manns[anim_manns[0]], choose_manns[anim_manns[1]], anim_area.replace(" ","_"))

    if path.isdir(anim_path) == False:
        mkdir(anim_path)
        copy_tree("_plots", anim_path)
    print('Figures for animation saved to:', anim_path)
        
    print('\nMannings n = {} on left animation, and n = {} on right '\
          'animation'.format(choose_manns[anim_manns[0]], choose_manns[anim_manns[1]]))
    
    return show_anim(anim, form)

### Import gauge data

In [ ]:
def gauge_data():
    from setplot import setplot
    plotdata = setplot()
    # reset variables
    h = [None] * (amount_manns)
    u = [None] * (amount_manns)
    v = [None] * (amount_manns)
    mu = [None] * (amount_manns)
    mv = [None] * (amount_manns)
    t = [None] * (amount_manns)
    eta = [None] * (amount_manns)

    for k in range(amount_manns):
        plotdata.outdir = '_output_{}'.format(k + 1)
        # get gauge data
        g = plotdata.getgauge(gauge_num)
        t[k] = g.t / 3600.  # convert to hours
        mu[k] = g.q[1,:] # u = east-west momentum
        mv[k] = g.q[2,:] # v = north-south momentum
        eta[k] = g.q[3,:]   # eta = h + B (depth plus bathymetry)
        h[k] = g.q[0,:]       # water depth
        u[k] = g.q[1,:] / h[k]  # u = east-west velocity
        v[k] = g.q[2,:] / h[k]  # v = north-south velocity
        
    if gtype == 'lagrangian':
        dis_lat = [None] * amount_manns
        dis_lon = [None] * amount_manns
        for j in range(amount_manns):
            dis_lat[j] = empty([len(mu[j])])
            dis_lon[j] = empty([len(mv[j])])
            for i in range(len(mu[j])):       
                dis_lat[j][i] = (det[list(dict_gauge).index(gauge_name)][1] - mu[j][i]) * 110045
            for h in range(len(mv[j])):   
                dis_lon[j][h] = (det[list(dict_gauge).index(gauge_name)][2] - mv[j][h]) * 87870.18
    else:
        dis_lat = 0
        dis_lon = 0
    
    return t,eta,u,v,mu,mv,dis_lat,dis_lon

### Parameterise graph

In [ ]:
def plotter_details(measure):
    if measure == 'height':
        var = eta
        plot_amount = 1
        direct = ['','']     
        yunit = 'meters'
        heading = 'Sea surface elevation at {} gauge'.format(gauge_name)

    elif measure == 'velocity':
        var = [u,v]
        plot_amount = 2
        direct = ['east-west', 'north-south']
        yunit = 'meters per second'
        heading = 'velocity at {} gauge'.format(gauge_name)
    
    elif measure == 'momentum':
        var = [mu,mv]
        plot_amount = 2
        direct = ['east-west', 'north-south']
        yunit = 'kilograms per meter per second'
        heading = 'momentum at {} gauge'.format(gauge_name)
        
    elif measure == 'travel':
        var = [dis_lat, dis_lon]
        plot_amount = 2
        direct = ['east-west', 'north-south']
        yunit = 'meters'
        heading = 'displacement of {} gauge'.format(gauge_name)
        
    return var, plot_amount, direct, yunit, heading

### Plot graph

In [ ]:
def plotter(measure):
    if gtype == 'lagrangian' and measure != 'travel':
        print('skipping plot, as gauge type must be "stationary" to plot {}'.format(measure))
        skip = True
    elif gtype == 'stationary' and measure == 'travel':
        print('skipping plot, as gauge type must be "lagrangian" to plot {}'.format(measure))
        skip = True
    else:
        skip = False
    
    if skip == False:
        var, plot_amount, direct, yunit, heading = plotter_details(measure)
        if plot_amount == 1:
            grid()
            for k in range(amount_manns):
                plot(t[k],var[k], linestyle = line[k][0], color = line[k][1], label="Manning's n = {:.3f}".format(manns[k]))
            title('{}'.format(heading))

        elif plot_amount == 2:
            for l in range(2):
                subplot(1, 2, (l+1))
                grid()
                for k in range(amount_manns):
                    plot(t[k], var[l][k], linestyle = line[k][0], color = line[k][1], label="Manning's n = {:.3f}".format(manns[k]))
                title('{} {}'.format(direct[l], heading))

        if gtype == 'lagrangian':
            ticklabel_format(useOffset = False, style='plain')
        legend()
        xlabel('Hours since earthquake')
        ylabel(yunit)
        

        tight_layout()
        gcf().set_size_inches(14,6)
        savefig('_saved/_plots/manns_{}/{}-gauge_{}.png'.format(manns, gauge_name.replace(' ','_'), measure))
        show()

In [ ]:
def plotter_compare():
    if gtype == 'lagrangian':
        print('Comparison can only be performed on "stationary" guage type')
    else:
        ax = gca()

        xmin, xmax, ymin, ymax = (3, 7, -0.20, 0.30)

        for k in range(amount_manns):
            ax.plot(t[k],eta[k], linestyle = line[k][0], color = line[k][1], label="Manning's n = {:.3f}".format(manns[k]))
        ax.plot(0,0,color='k', label="Real life data")
        ax.legend()
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_xlabel('Hours since earthquake')
        ax.set_ylabel('Meters')
        ax.set_title('Sea surface elevation at gauge 32412')

        ax.set_zorder(2)
        ax.set_facecolor('none')

        ax_tw_x = ax.twinx()
        ax_tw_x.axis('off')
        ax2 = ax_tw_x.twiny()

        im = imread('gauge32412-actual-detide.png')
        ax2.imshow(im, extent=[xmin, xmax, ymin, ymax], aspect='auto')
        ax2.axis('off')

        gcf().set_size_inches(14,6)
        savefig('_saved/_plots/manns_{}/{}-compare_gauge_height.png'.format(manns, gauge_name.replace(' ','_')))
        show()

## Custom parameters

### Manning's values

If true, Manning values will be a family, if false, they will be custom

In [ ]:
#vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
family_mann = False
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

#### Family of Manning's values

Assign Manning's values as a family centered around a middle value. 

10 curves may take from 5-10 minutes to plot dependent on your machine.

In [ ]:
#vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
amount_manns = 10
mid_mann = 0.5
space_manns = 0.1
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

#### Custom Manning's values

Dictionary of custom mannings values

In [ ]:
#vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
dict_mann = {'test':5,
            'geoclaw_tsunami':0.025,
            'yanigasawa_koshimura_2009_mangrove':0.04,
            'koshimura_2009_low_urban':0.053,
            'koshimura_2009_med_urban':0.094,
            'koshimura_2009_high_urban':0.172,
            'kotani_1998_coast':0.025,
            'kotani_1998_farmland':0.02,
            'kotani_1998_forest':0.03
           }
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
for key in dict_mann:
    if (len(dict_mann) - 1) != list(dict_mann).index(key):
        print("'" + key + "'" + ",")
    else:
        print("'" + key + "'")

Custom manning's values to plot (if not intending to plot a family of curves): simply run the two cells above, then copy and paste the selected Manning's values you wish to use from the output.

In [ ]:
#vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
cust_manns = ['test',
            'kotani_1998_forest',
            'koshimura_2009_med_urban'
            ]
 #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

### Gauges

custom locations to plot gauges at. If unsure, just copy and past this, and replace the named parts (try to keep location description under 35 characters):

'location description':[number(0-999) , latitide, longitude, time start(default to 0.), time steps(default to 1.e10)]

hint: area wave resides in first hour [-85,-72,-38,-25]

In [ ]:
#vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
dict_gauge = {'123':[123, -77.3, -12.3, 3.5*3600, 1.e10],
            '32412':[32412, -86.392, -17.975, 3.5*3600, 1.e10],
            'south west epicenter':[0,-76,-40, 0., 1.e10],
            'epicenter':[1,-72.733,-34.909, 0., 1.e10], 
            'Alejandro Selkirk Island west coast': [2,-80.756765, -33.767759, 0., 1.e10], 
            'Robinson Crusoe Island west coast':  [3,-78.770779, -33.668985, 0., 1.e10],
           }
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

gtype is gauge type, which can either be 'stationary' or 'lagrangian'

In [ ]:
#vvvvvvvvvvvvvvvvvvvvvvvv
gtype = 'lagrangian'
#^^^^^^^^^^^^^^^^^^^^^^^^

## Initiate parameter choice

Note : All manning values are shown in their absolute value (the program does not display negative values any differently from positive ones)

In [ ]:
choose_manns = {}

if family_mann == True:
    manns = [None] * amount_manns

        
elif family_mann == False: 
    manns = [None] * len(cust_manns)
    for k in range(len(cust_manns)): 
        manns[k] = dict_mann[cust_manns[k]]
    manns.sort()
        
amount_manns = len(manns)
    
for k in range(amount_manns):
    if family_mann == True:
        if amount_manns%2 == 0:
            manns[k] = abs(mid_mann + (k + 1 - (amount_manns)/2) * space_manns)
        else:
            manns[k] = abs(mid_mann + (k + 1 - (amount_manns + 1) /2) * space_manns)
            
    choose_manns[k+1] = manns[k] 
    
    copyfile("setrun.py", "setrun_{}.py".format(k + 1))

In [ ]:
det = [None] * len(dict_gauge)

for key in dict_gauge:
    k = list(dict_gauge).index(key)
    det[k] = dict_gauge[key]

# Animations

## Animation set up

In [ ]:
anim_manns = [None] * 2
if len(choose_manns) > 2:
    anim_manns[0] = int(input('Choose (1-{}) the first mannings values to'\
                          ' compare via animation: {}\n'.format(amount_manns, choose_manns)))
    anim_manns[1] = int(input('Choose (1-{}) the second mannings values to'\
                          ' compare via animation: {}\n'.format(amount_manns, choose_manns)))
else:
    print('Mannings values to be compiled : {}\n'.format(manns))
    anim_manns[0] = 1
    anim_manns[1] = 2

In [ ]:
anim_dir = '_saved/_anim/manns_[{},{}]'.format(choose_manns[anim_manns[0]],choose_manns[anim_manns[1]])
if path.isdir(anim_dir) == False:
    mkdir(anim_dir)

## North west coast

In [ ]:
anim_manns = init_comp(2, anim123 = True) 
stitch_anim('North west coast', 'js')

## Full map

You can use this animation to check your gauges are positioned correctly. Custom gauge numbers correspond to their position in the dictionary above.

In [ ]:
anim_manns = init_comp(1)
stitch_anim('Full map', 'js')

# Plots

## Plot set-up

In [ ]:
# line-styler
rainbow = ['#ff0000', '#c3ff00', '#00ff00', '#00ffff', 
          '#0000ff', '#ff00ff']
morse = ['-','-.','--', '.']

line = [None] * amount_manns

for k in range(amount_manns):
    line[k] = [None] * 2
    j = k % len(rainbow)
    l = int(str(k/len(rainbow))[:1]) 
    line[k][0] = morse[l]
    line[k][1] = rainbow[j]

In [ ]:
if path.isdir('_saved/_plots/manns_{}'.format(manns)) == False:
    mkdir('_saved/_plots/manns_{}'.format(manns))

## Initiate gauges

In [ ]:
exno = 2

if exno == 1:
    raise Exception("You need to run exercise number 2 with all gauge plots"\
                    " compiled to view the plots correctly. Please run the"\
                    " cell above this error and then continue running the notebook")

init_comp(exno, plots = True, gtype = gtype)

## Plot gauges

In [ ]:
print('All plots saved to _saved/_plots/manns_{}'.format(manns))
for key in dict_gauge:

    gauge_name = key
    gauge_num = dict_gauge[gauge_name][0]
    t,eta,u,v,mu,mv,dis_lat,dis_lon = gauge_data()
    print('#################-{}-#################################'.format(gauge_name))
    if gtype == 'stationary':
        plotter('velocity')
        plotter('height')
        if gauge_num == 32412:
            plotter_compare()
        plotter('momentum')
    if gtype == 'lagrangian': 
        plotter('travel')

# TESTING AREA

Notes :
- Save plots and animations (can I use the anim. in the first function? like anim.save?)
- Write figure legends
- Use G123 data to test different values given in the paper for areas such as forests,
    importance of mangrove trees was undervalued
- colours need to have more distinction

attempting to plot lagrangian guages which start at 0, and show travel from tsunami (not speed, distance)

In [ ]:
# def plotter(measure):
#     if measure == 'height':
#         grid()
#         for k in range(amount_manns):
#             plot(t[k],eta[k], linestyle = line[k][0], color = line[k][1], label="Manning's n = {:.3f}".format(manns[k]))
#         legend()
#         xlabel('Hours since earthquake')
#         ylabel('meters')
#         title('Sea surface elevation at gauge {}'.format(gauge_name))

#     else: 
#         if measure == 'velocity':
#             var = [u, v]
#             add_unit = ['', 's']
            
#         elif measure == 'momentum':
            
#             if gtype == 'lagrangian':
#                 for j in range(amount_manns):
# #                     mu[j] = [(x - det[j][1]) * 110045 for x in mu[j]]
# #                     mv[j] = [(y - det[j][2]) * 87870.18 for y in mv[j]]
#                     for i in range(len(mu[j])):
#                         mu[j][i] = det[list(dict_gauge).index(gauge_name)][1] - mu[j][i]
#                     for h in range(len(mv[j])):    
#                         mv[j][i] = det[list(dict_gauge).index(gauge_name)][2] - mv[j][i]
#                 print(mu[0][0],mu[1][0],mv[0][0],mv[1][0])
#             var = [mu, mv]
#             add_unit = ['kilograms per ', '']
#         direct = ['east-west ', 'north-south ']
#         for l in range(2):
#             subplot(1, 2, (l+1))
#             grid()
#             for k in range(amount_manns):
#                 plot(t[k], var[l][k], linestyle = line[k][0], color = line[k][1], label="Manning's n = {:.3f}".format(manns[k]))
#             if gtype == 'lagrangian':
#                 ticklabel_format(useOffset = False, style='plain')
#             xlabel('Hours since earthquake')
#             ylabel('{}meter{} per second'.format(add_unit[0], add_unit[1]))
#             title('{}{} at {}'.format(direct[l], measure, gauge_name))
    
#     tight_layout()
#     gcf().set_size_inches(14,6)
#     savefig('_saved/_plots/manns_{}/gauge_{}-measure_{}.png'.format(manns, gauge_name, measure))
#     show()

In [ ]:
# anim = animation_tools.animate_from_plotdir('_saved/_anim/ex_1-fig_0-manns_[0.025,0.04]', 0)
# show_anim(anim)

In [ ]:
# def not_tall():
#     direct = ['east-west ', 'north-south ']
#     for j in range(2):
#         subplot(1, 2, (j+1))
#         grid()
#         for k in range(amount_manns):
#             plot(t[k], var[j][k], linestyle = line[k][0], color = line[k][1], label="Manning's n = {:.3f}".format(manns[k]))
#         xlabel('Hours since earthquake')
#         ylabel('{}meter{} per second'.format(add_unit[0], add_unit[1]))
#         title('{}{} at {}'.format(direct[j], measure, gauge_name))

# def plotter():
#         grid()
#         for k in range(amount_manns):
#             plot(t[k],eta[k], linestyle = line[k][0], color = line[k][1], label="Manning's n = {:.3f}".format(manns[k]))
#         legend()
#         xlabel('Hours since earthquake')
#         ylabel('meters')
#         title('Sea surface elevation at gauge {}'.format(gauge_name))
 
#         var = [u, v]
#         add_unit = ['', 's']
#         not_tall()

#         var = [mu, mv]
#         add_unit = ['kilograms per ', '']
#         not_tall()
    
    
#         tight_layout()
#         gcf().set_size_inches(14,6)
#         savefig('_saved/_plots/manns_{}/gauge_{}-measure_{}.png'.format(manns, gauge_name, measure))
#         show()

In [ ]:
# from PIL import Image
# from distutils.dir_util import copy_tree
# if path.isdir('_saved') == False:
#     mkdir('_saved')
# def stitch_save_anim(choose_manns, anim_manns, exno = 1, figno = 0):
#     system("mkdir _plots")
    
#     if exno == 1:
#         frames = 5
#         extra = ''
#     elif exno == 2:
#         frames = 19
#         extra = 0
#     else:
#         print ("Error : Enter the exercise you are running, either 1, or 2")
        
#     for a in range(exno - 1 ,frames + 1):
#         im = Image.open('_plots_{}/frame{:04d}fig{}.png'.format(anim_manns[0],a, figno))
#         im_alt = Image.open('_plots_{}/frame{:04d}fig{}.png'.format(anim_manns[1], a, figno))
#         dst = Image.new('RGB', (im.width + im_alt.width, im.height))
#         dst.paste(im, (0, 0))
#         dst.paste(im_alt, (im.width, 0))
#         dst.save('_plots/frame{:04d}fig{}.png'.format(a, figno))
#         source = '_plots/'
#         dest = '_saved/mann_{}_{}-ex{}-fig{}'.format(choose_manns[anim_manns[0]]\
#                                                      ,choose_manns[anim_manns[1]],1,0)
#         copy_tree(source, dest)
        
# #         animation_tools.animate_from_plotdir('_plots', figno)
# #     print('\nMannings n = {} on left animation, and n = {} on right '\
# #           'animation'.format(choose_manns[anim_manns[0]], choose_manns[anim_manns[1]]))
# #     return show_anim(anim)
                  
# for j in range (3):
#     for k in range (amount_manns):
#         mod = importlib.import_module("setrun_{}".format(k+1))
#         rundata = mod.setrun()
#         # Variables to change
#         #vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
#         # Change Mannings
#         rundata.geo_data.manning_coefficient = manns[k]
#         # Change output times:
#         if j == 2:
#             rundata.clawdata.output_style = 2 #This is the difference from ex1 to ex2 in the animation
#             rundata.clawdata.output_times = linspace(3.5,8,19) * 3600.
#             # Change gauge location:
#             from clawpack.amrclaw.data import GaugeData
#             rundata.replace_data("gaugedata", GaugeData())  # clear old gauge data
#             rundata.gaugedata.gauges = []   # empty list
#             rundata.gaugedata.gauges.append([32412, -86.392, -17.975, 0., 1.e10]) # original gauge
#             rundata.gaugedata.gauges.append([123, -77.3, -12.3, 3.5*3600, 1.e10]) # relocated gauge
#         #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#         compiler(k, amount_manns)
#     if j < 2:
#         stitch_save_anim(choose_manns, anim_manns,1,j)
#     else:
#         stitch_save_anim(choose_manns, anim_manns,2,1)